In [2]:
import json
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
from re import sub

In [5]:
path_to_chromeDriver = '/Users/drahcir1/Documents/chromedriver'
url ='https://www.idaholottery.com/games/scratch'

# disables chrome from loading images 
# https://stackoverflow.com/questions/28070315/python-disable-images-in-selenium-google-chromedriver
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

#creates a selenium chrome driver
driver = webdriver.Chrome(path_to_chromeDriver,chrome_options=chrome_options)
driver.implicitly_wait(5)
driver.get(url)
soup = BeautifulSoup(driver.page_source)

driver.close()

<ipython-input-5-8a31c2a34eed>:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(path_to_chromeDriver,chrome_options=chrome_options)


In [94]:
games = soup.findAll("div",{"class":"game__content"})

In [96]:
def stringToFloat(s):
     return float(sub(r'[^\d.]', '', s))
def getGameName(gameTag):
    return gameTag.h5.text

def getGameCost(gameTag):
    string = gameTag.find("span", {"game__info-price"}).text
    return stringToFloat(string)

def getGameRowPrize(rowTag):
    priceString = rowTag.find("td",{"class":"prizes-prize"}).text
    return stringToFloat(priceString)

def getGameRowPrizesRemaining(rowTag):
    remainingString = rowTag.find("td",{"class":"prizes-remaining"}).text
    return stringToFloat(remainingString)  

def getGameRowData(gameTag):
    output = []
    tbody = gameTag.tbody
    for row in tbody.findAll('tr'):
        output.append({"PrizeAmount":getGameRowPrize(row),
                       "PrizesRemaining":getGameRowPrizesRemaining(row)})
    return output

In [97]:
today = datetime.today().strftime('%Y%m%d')
games_data = {"date":today}
gamesContent = []
for gameTag in games:
    game_data = {}
    game_data["name"] = getGameName(gameTag)
    game_data["cost"] = getGameCost(gameTag)
    game_data["data"] = getGameRowData(gameTag)
    gamesContent.append(game_data)
    
    
games_data['data'] = gamesContent
games_data



{'date': '20210410',
 'data': [{'name': 'Mystery Box Multiplier Cashword',
   'cost': 10.0,
   'data': [{'PrizeAmount': 100000.0, 'PrizesRemaining': 2.0},
    {'PrizeAmount': 5000.0, 'PrizesRemaining': 3.0},
    {'PrizeAmount': 1000.0, 'PrizesRemaining': 6.0},
    {'PrizeAmount': 500.0, 'PrizesRemaining': 157.0},
    {'PrizeAmount': 250.0, 'PrizesRemaining': 1130.0},
    {'PrizeAmount': 100.0, 'PrizesRemaining': 5623.0},
    {'PrizeAmount': 50.0, 'PrizesRemaining': 8586.0},
    {'PrizeAmount': 30.0, 'PrizesRemaining': 8926.0}]},
  {'name': 'Fox Fortune Doubler',
   'cost': 5.0,
   'data': [{'PrizeAmount': 50000.0, 'PrizesRemaining': 2.0},
    {'PrizeAmount': 1000.0, 'PrizesRemaining': 3.0},
    {'PrizeAmount': 500.0, 'PrizesRemaining': 5.0},
    {'PrizeAmount': 200.0, 'PrizesRemaining': 10.0},
    {'PrizeAmount': 150.0, 'PrizesRemaining': 101.0},
    {'PrizeAmount': 100.0, 'PrizesRemaining': 1632.0},
    {'PrizeAmount': 50.0, 'PrizesRemaining': 5521.0}]},
  {'name': 'Limited 8',
   'co

In [ ]:

with open(f'data/idaho{today}_scratchers.json', 'w') as fp:
    json.dump(games_data, fp)    
